## 重回帰分析

- データと手法の概要
- モデル作成と母数の推定・診断
    - 多重共線性のチェック
- モデルの評価と解釈
    - 決定係数によるモデル全体の評価
    - 切片と偏回帰係数の解釈
    - 偏回帰係数の信頼区間
    - 単位の異なる説明変数が混在する場合の標準偏回帰係数の算出
- 報告例
- 質的変数を含む重回帰分析
- AICとBICによるモデルの評価
- 重回帰分析と母数推定論
    - 最小二乗法による母数推定の概要
        - 自由度調整済み決定係数とAICの意義
    - 最尤法による母数推定の概要
- 偏回帰係数の解釈
- 決定係数とその検定
- 切片と偏回帰係数の検定
- 切片と偏回帰係数の信頼区間
- VIFの理論

In [1]:
#markdownの表を左寄せするためにIDdisplayパッケージでhtml,cssを適用できるようにする
library(IRdisplay)
IRdisplay::display_html('<style> table { float:left; } </style>')

### データと手法の概要

#### 用いる仮想データ:顧客満足度データ
データ概要:都内を中心に30店舗を展開しているフィットネスジムチェーンの顧客満足度のアンケート結果のデータ

変数名|変数の内容
:---:|:---:
店舗番号|各店舗にランダムに付与されたID
顧客数|各店舗の顧客数
立地満足度|立地に関する顧客満足度の平均
設備満足度|設備に関する顧客満足度の平均
店舗面積満足度|店舗面積に関する顧客満足度の平均
トレーナー満足度|トレーナーに関する満足度の平均
トレーナー数|各店舗のトレーナー数
接客件数|1=研修あり、0=なし
入会特典|1=特典あり、0=なし

### 回帰分析の具体的な手順
- 回帰モデルの作成
- モデルにおける母数の推定
- モデルの診断と評価
- 偏回帰係数の解釈

### モデル作成と母数の推定・診断

In [2]:
#データの読み込み
csdat <- read.csv("https://raw.githubusercontent.com/gndb3168/Renshu_data/master/R_MA/Sec_03/CS.csv")
head(csdat, 3)

店舗番号,顧客数,立地満足度,設備満足度,店舗面積満足度,トレーナー満足度,トレーナー数,接客研修,入会特典
1,595,4.2,5.4,5.9,6.8,12,0,1
2,483,4.7,4.1,3.6,6.1,11,0,0
3,601,7.5,6.3,5.5,5.6,11,1,0


In [3]:
#重回帰分析の実行

#顧客数を目的変数、立地満足度、設備満足度、店舗面積満足度が説明変数
res1 <- lm(顧客数~立地満足度+設備満足度+店舗面積満足度+トレーナー満足度, data=csdat)

#結果の出力
summary(res1)


Call:
lm(formula = 顧客数 ~ 立地満足度 + 設備満足度 + 店舗面積満足度 + 
    トレーナー満足度, data = csdat)

Residuals:
    Min      1Q  Median      3Q     Max 
-63.636 -36.548  -0.314  21.419 101.890 

Coefficients:
                 Estimate Std. Error t value Pr(>|t|)    
(Intercept)       -35.204     50.659  -0.695  0.49351    
立地満足度         29.105      5.684   5.120 2.73e-05 ***
設備満足度         21.640      7.036   3.076  0.00503 ** 
店舗面積満足度     23.803      6.971   3.414  0.00219 ** 
トレーナー満足度   32.421      5.608   5.781 5.01e-06 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 45.11 on 25 degrees of freedom
Multiple R-squared:  0.8248,	Adjusted R-squared:  0.7968 
F-statistic: 29.43 on 4 and 25 DF,  p-value: 3.957e-09


### 推定結果の診断(多重共線性のチェック)
#### 「多重共線性」とは?

- 説明変数間の相関係数の絶対値が大きいと偏回帰係数の標準誤差を正しく推定できない

In [4]:
#説明変数間の相関係数
cor(csdat$トレーナー満足度, csdat$トレーナー数)

[1] 0.9843212

In [5]:
#説明変数間の相関が高い場合の重回帰分析の出力

#単回帰分析
resm1 <- lm(顧客数~トレーナー満足度, data=csdat)
summary(resm1)

#重回帰分析
resm2 <- lm(顧客数~トレーナー満足度+トレーナー数, data=csdat)
summary(resm2)


Call:
lm(formula = 顧客数 ~ トレーナー満足度, data = csdat)

Residuals:
     Min       1Q   Median       3Q      Max 
-155.049  -50.396    0.174   41.746  202.362 

Coefficients:
                 Estimate Std. Error t value Pr(>|t|)    
(Intercept)        314.04      54.69   5.743 3.67e-06 ***
トレーナー満足度    37.09      10.46   3.544  0.00141 ** 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 84.62 on 28 degrees of freedom
Multiple R-squared:  0.3097,	Adjusted R-squared:  0.285 
F-statistic: 12.56 on 1 and 28 DF,  p-value: 0.001405



Call:
lm(formula = 顧客数 ~ トレーナー満足度 + トレーナー数, data = csdat)

Residuals:
     Min       1Q   Median       3Q      Max 
-157.495  -46.837   -0.332   37.689  208.062 

Coefficients:
                 Estimate Std. Error t value Pr(>|t|)
(Intercept)        373.19     260.79   1.431    0.164
トレーナー満足度    50.88      60.35   0.843    0.407
トレーナー数       -12.83      55.26  -0.232    0.818

Residual standard error: 86.08 on 27 degrees of freedom
Multiple R-squared:  0.3111,	Adjusted R-squared:   0.26 
F-statistic: 6.095 on 2 and 27 DF,  p-value: 0.006538


トレーナー満足度のみの回帰では標準誤差が10.46であるが、トレーナー数を加えた重回帰では60.35と約6倍になり、推定精度が悪くなっている。

#### 多重共線性の検出
- 説明変数の相関行列を考察する
- VIF(variance inflation factor; 分散拡大要因)という指標で客観的に診断する
    - <span style="color:red;">VIFが10を超えていたら多重比較が生じている可能性</span>

In [6]:
#VIFを算出するための前準備(carパッケージ)
library(car)    

Loading required package: carData


In [7]:
#VIFの算出(carパッケージ)
vif(resm2)    #回帰分析の結果を引数にする

トレーナー満足度     トレーナー数 
        32.14226         32.14226

VIFが10を超えているので多重共線性を生じている可能性が高い。  
どちらか一方の説明変数を削除してモデルを再構築すると良い。

In [8]:
#顧客満足度の4説明変数に関するVIFの算出
vif(res1)

立地満足度       設備満足度   店舗面積満足度 トレーナー満足度 
        1.042106         1.589938         1.548752         1.010455

どの説明変数もVIFが10を大きく下回っているので、多重共線性の影響は小さいと考えられる。

### モデルの評価と解釈
- 決定係数によるモデル全体の評価
- 切片と偏回帰係数の解釈
- 偏回帰係数の信頼区間
- 単位の異なる説明変数が混在する場合(標準偏回帰係数の算出)

### 決定係数によるモデル全体の評価

#### 決定係数とは?
- 目的変数の分散のうち、説明変数で説明できた割合(分散説明率)
- $R^2$で表現され、割合なので1に近いほど説明変数で目的変数を良く説明できているといえる
- 偏回帰係数の解釈の前に確認すべき指標

#### 自由度調整済み決定係数とは?
- 決定係数は説明変数の数を増やすだけで値が向上する性質
- 説明変数が多くなるほど決定係数を下方修正する指標
- 決定係数に対して自由度調整済み決定係数が極端に小さい場合、目的変数に寄与しない説明変数を投入しすぎている可能性
```R
F-statistic: 29.43 on 4 and 25 DF, p-value:3.957-09
```
は、母集団において「偏回帰係数$\beta$がすべて0」という帰無仮説に対するF検定の結果  
帰無仮説が採択されると決定係数が0になるので、決定係数の有意性検定ともいわれる

### 切片と偏回帰係数の解釈

In [9]:
#切片と偏回帰係数の確認
summary(res1)$coefficients

,Estimate,Std. Error,t value,Pr(>|t|)
(Intercept),-35.20449,50.658748,-0.694934,4.935055e-01
立地満足度,29.10529,5.684336,5.120262,2.731219e-05
設備満足度,21.63968,7.035670,3.075709,5.030246e-03
店舗面積満足度,23.80296,6.971247,3.414448,2.186653e-03
トレーナー満足度,32.42134,5.607780,5.781493,5.014765e-06


- 切片(intercept)は説明変数の値がすべて0のときの期待値(解釈することはまずない(笑))  
- Estimateは偏回帰係数であり、他の説明変数を一定にし、ある説明変数の値だけ1増加させたときの目的変数に期待される変化量
- 「他の説明変数の値を一定にした」という前提のもとで、顧客数に最も寄与するのは「トレーナー満足度」と解釈できる
- Std.errorは推定値の標準誤差
- t-valueは「母集団において当該偏回帰係数が0」を帰無仮説とした$t$検定に対する$t$の値
- Pr(>|t|)はt値に対する$p$値

### 偏回帰係数の信頼区間
重回帰分析では、偏回帰係数の有意性だｋでなく、推定値の大きさにも興味があることが多いので信頼区間を合わせて報告すること

In [10]:
#切片と偏回帰係数の信頼区間の算出
confint(res1, level=0.95)    #confint(lmオブジェクト, 区間水準)

,2.5 %,97.5 %
(Intercept),-139.538132,69.12916
立地満足度,17.398182,40.81240
設備満足度,7.149443,36.12991
店舗面積満足度,9.445408,38.16051
トレーナー満足度,20.871903,43.97078


偏回帰係数が0という帰無仮説は棄却されているものの、信頼区間の幅が広く、全体的に係数の推定制度は全体的に低め

### 単位の異なる説明変数が混在する場合(標準偏回帰係数の算出)
- 偏回帰係数は変数の単位に影響される
- 「立地満足度」は10段階評価、「トレーナー数」は人数と単位が異なる場合、係数の比較が難しくなる
- <span style="color:red;">単位の異なる説明変数をモデルに含める場合には、すべての変数をz得点に変換し、偏回帰係数から単位の影響を取り除くと良い</span>
- 標準化したデータから推定された偏回帰係数を「標準偏回帰係数」と呼ぶ
- 標準偏回帰係数はおおよそ-1~+1の間の値を取り、影響の強さを解釈しやすくなる

In [11]:
#標準偏回帰係数の算出

#データの標準(scale)とデータフレーム化(as.data.frame)
scsdat <- as.data.frame(scale(csdat))

#標準偏回帰係数の推定
res2 <- lm(顧客数~立地満足度+トレーナー数, data=scsdat)
summary(res2)


Call:
lm(formula = 顧客数 ~ 立地満足度 + トレーナー数, data = scsdat)

Residuals:
     Min       1Q   Median       3Q      Max 
-1.38531 -0.41534  0.01021  0.29706  2.09443 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  1.544e-16  1.398e-01   0.000 1.000000    
立地満足度   4.020e-01  1.422e-01   2.828 0.008729 ** 
トレーナー数 5.317e-01  1.422e-01   3.739 0.000878 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.7655 on 27 degrees of freedom
Multiple R-squared:  0.4545,	Adjusted R-squared:  0.4141 
F-statistic: 11.25 on 2 and 27 DF,  p-value: 0.0002799


- データを標準化した場合、回帰モデルの切片は理論的に0になるので、切片の情報は不要
- 標準化したとしても、決定係数の値やモデルの適合に関するF検定、偏回帰係数に関する$t$検定の結果は変わらない

### 報告例
　「顧客数」に対する「立地満足度」「設備満足度「店舗面積満足度」「トレーナー満足度」の影響を検討するため、「顧客数」を目的変数、顧客満足度の4変数を説明変数とした重回帰分析っを行った。VIFによって多重共線性の可能性を検討したところ、すべての説明変数において2未満であり、多重共線性が生じている可能性は低いと判断した。  
 　モデルの決定係数$R^2$は.825、自由度調整済み決定係数は.797であり、「顧客数」の分散の約8割を顧客満足度の4変数が説明する結果となった。下表にモデル中の母数と決定係数の推定値、そして母数の信頼区間を示す。
<div style="text-align: center;">
    表1.重回帰分析の結果
</div>
\begin{array}{lrrr} \hline
	& 推定値 & 標準誤差 & 95\%CI \\ \hline 
			切片 & -35.204 & 50.659 & [-139.538, 69.129] \\
			立地満足度 & 29.105 & 5.684 & [17.398, 40.812] \\
			設備満足度 & 21.640 & 7.036 & [7.149, 36.130] \\
			店舗面積満足度 & 23.803 & 6.971 & [9.445, 38.161] \\
			トレーナー満足度 & 32.421 & 5.608 & [20.872, 43.971] \\ \hline
            R^2 = .825^{* * *}
\end{array}
　決定係数に関するF検定は0.1%水準で有意であり(F(4.25)=29.43, p<.001)、母集団において顧客満足度の4変数の少なくとも1つは「顧客数」を説明している可能性があることが示唆された。また、95%信頼区間より、すべての説明変数において偏回帰係数は少なくとも5%水準で有意であり、母集団における存在が示唆される結果となった。  
　説明変数の単位が等しいので、偏回帰係数を解釈する。推定値が最大であったのは「トレーナー満足度」の32.421(95%CI[20.872, 43.971]であった。他の説明変数を一定にしたうえで「トレーナー満足度」を1店増加させると、32.421人の顧客数が見込まれる結果となった。一方、推定値が最小であったのは「設備満足度」の21.640であり(95%CI[7.149, 36.130])、他の説明変数を一定にしたうえで「設備満足度」を1点増加させると、21.640人の顧客の増加が見込まれる結果となった。 

#### 補足
<div style="color: red">
mathjaxはtabular環境に対応していないため、array環境で代用した。   
latexコードでセンタリング等も出来なかったため、通常の文字の中央寄せはhtmlで対応。
</div>

[latex tabular environment](https://github.com/mathjax/mathjax-docs/wiki/LaTeX-Tabular-environment)  
[html,tex対応表](http://www.ne.jp/asahi/hishidama/home/tech/tex/texhtml.html)  
[easy copy mathjax](http://easy-copy-mathjax.xxxx7.com/#table)  
[style属性でcssを複数指定する方法](http://w-d-l.net/css__course__inline_style/)

### 質的変数を含む重回帰分析

In [12]:
#質的変数を含む重回帰分析
res3 <- lm(顧客数~立地満足度+設備満足度+店舗面積満足度+トレーナー満足度+接客研修+入会特典, data=csdat)
summary(res3)


Call:
lm(formula = 顧客数 ~ 立地満足度 + 設備満足度 + 店舗面積満足度 + 
    トレーナー満足度 + 接客研修 + 入会特典, data = csdat)

Residuals:
    Min      1Q  Median      3Q     Max 
-78.543 -25.702  -2.392  17.573  91.820 

Coefficients:
                 Estimate Std. Error t value Pr(>|t|)    
(Intercept)       -13.878     48.906  -0.284  0.77913    
立地満足度         27.488      5.544   4.958 5.18e-05 ***
設備満足度         24.745      6.779   3.650  0.00134 ** 
店舗面積満足度     17.963      7.067   2.542  0.01822 *  
トレーナー満足度   28.515      5.592   5.099 3.65e-05 ***
接客研修            8.014     16.121   0.497  0.62383    
入会特典           40.128     18.065   2.221  0.03646 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 42.55 on 23 degrees of freedom
Multiple R-squared:  0.8566,	Adjusted R-squared:  0.8192 
F-statistic:  22.9 on 6 and 23 DF,  p-value: 1.27e-08


決定係数が量的変数のみの分析では0.8248であったのに対して3%程度分散説明率が上がっている。  
入会特典を実施している店舗(入会特典=1)の店舗の方が実施していない店舗(入会特典=0)よりも40.128人顧客数が多いことが期待される。

### AICとBICによるモデルの評価
説明変数が異なるモデル間で適合度を比較する場合に用いる。値が小さいモデルが適合が良好。
- AIC:Akaike information criterion
- BIC:Bayesian information crierion

In [13]:
#AICの算出

#説明変数が量的変数のみの回帰モデル
extractAIC(res1)

#説明変数に質的変数も含めた回帰モデル
extractAIC(res3)

[1]   5.0000 233.0808

[1]   7.0000 231.0751

出力結果1つ目の要素がモデル中の母数の数、2つ目の要素がAIC。  

In [14]:
#BICの算出

#BICはextractAICの引数kに観測対象の数の自然対数を指定する(デフォルトはk=2)
extractAIC(res1, k=log(30))

#説明変数に質的変数も含めた回帰モデル
extractAIC(res3, k=log(30))

[1]   5.0000 240.0868

[1]   7.0000 240.8835

AICとBICの結果から、実質的には両モデルの適合の良さはほぼ同じ。

### 決定係数とその検定

#### 目的変数$y$の平方和$SS_y$
$$ SS_y = \sum^N_{i=1}(y_i - \bar{y})^2 $$

#### 予測値$\hat{y}$の平方和$SS_{\hat{y}}$
$$ SS_y = \sum^N_{i=1}(\hat{y}_i - \bar{\hat{y}})^2 $$

#### 誤差の平方和$SS_e$
$$ SS_e = \sum^N_{i=1} e_i^2 $$

3つの平方和の間で以下の関係式が成り立つ
$$ SS_y = SS_{\hat{y}} + SS_e $$

#### 決定係数$R^2$
$$ R^2 = \frac{SS_{\hat{y}}}{SS_y} $$

#### 自由度調整済み決定係数
$$ 1-\frac{SS_e / (N - J - 1)}{SS_y / (N-1)} $$

### 切片と偏回帰係数の検定

$$ t_{\alpha} = \frac{\alpha}{\alpha \mbox{の標準誤差}} $$
$$ t_{\beta} = \frac{\beta}{\beta \mbox{の標準誤差}} $$

これら検定統計量は自由度$N - J - 1$の$t$分布に従う

### VIFの理論

ある説明変数$x_j$を他の全ての説明変数で重回帰分析し、その決定係数を$R_j^2$とした場合、

$$ VIF_j = \frac{1}{1 - R_j^2} $$